In [4]:
import torch
features = {}
def hook_fn(module, input, output):
    features['pre_head'] = output.detach() 
#
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Init model

In [ ]:
# load eva02_large_patch14_clip_336 from timm
import timm
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = "hf_hub:timm/eva02_large_patch14_clip_336.merged2b_ft_inat21"
model = timm.create_model(model_path, pretrained=True)
# model = torch.nn.Sequential(*list(model.children())[:-1]).to(device)
model = model.to(device)

hook_handle = model.norm.register_forward_hook(hook_fn)
model.eval() 

In [2]:
# load ConvNeXt
import torch
from PDDD.Codes.modelpy.visual_model.ConvNeXt import convnext_base

# # Khởi tạo kiến trúc mô hình trước
model = convnext_base(6)

# # Load trọng số đã lưu
model_path = "/media/icnlab/Data/Manh/tinyML/PDDD/model/ConvNeXt.std"
model.load_state_dict(torch.load(model_path, map_location=device), strict=False)
model=model.to(device)
# model.eval()  # Đặt model ở chế độ inference

In [49]:
# load ViT_L, ViLT
import torch
from PDDD.Codes.modelpy.visual_model.ViT_L import VisionTransformer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Khởi tạo kiến trúc mô hình trước
model = VisionTransformer()

# Load trọng số đã lưu
model_path = "/media/icnlab/Data/Manh/tinyML/PDDD/model/ViT_L.std"
# model_path = "/media/icnlab/Data/Manh/tinyML/PDDD/model/ViLT.mdl"
model.load_state_dict(torch.load(model_path, map_location=device), strict=False)
model=model.to(device)

# hook_handle = classification_head.head[1].register_forward_hook(hook_fn)
ishook = True

# model.eval()  # Đặt model ở chế độ inference

In [5]:
# load ResNet152
import torch
import os
from PDDD.Codes.modelpy.visual_model.ResNet_50_101_152 import ResNet152

model = ResNet152()

# Load trọng số đã lưu
model_path = "/media/icnlab/Data/Manh/tinyML/PDDD/model/ResNet152.std"
model.load_state_dict(torch.load(model_path, map_location=device))
model=model.to(device)

# hook_handle = model.avgpool.register_forward_hook(hook_fn)
model_name = os.path.basename(model_path)

/home/icnlab/anaconda3/envs/aiot/lib/python3.11/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [7]:
# load MobileNetV3
import torch
import os
from PDDD.Codes.modelpy.visual_model.MobileNetV3 import mobilenet_v3_small, mobilenet_v3_large

model = mobilenet_v3_small()
model_path = '/media/icnlab/Data/Manh/tinyML/PDDD/model/MobileNetV3.std'
model.load_state_dict(torch.load(model_path, map_location=device))
model=model.to(device)

model_name = os.path.basename(model_path)

### Check model layer

In [ ]:
model.eval()

In [ ]:
import torch.nn as nn
backbone = nn.Sequential(*list(model.children()))  
backbone

In [ ]:
classification_head = dict(model.named_children())['fc']
classification_head

Linear(in_features=2048, out_features=1024, bias=True)

In [ ]:
print(model.layer4)
print(model.fc)

In [15]:
for name, param in model.named_parameters():
    if name.startswith(('conv1', 'layer1', 'layer2', 'layer3')):
        param.requires_grad = False
    else:
        param.requires_grad = True


num_trainable_params = sum(1 for p in model.parameters() if p.requires_grad)
print(f"Số lượng tensor cần training: {num_trainable_params}")

num_trainable_elements = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Tổng số tham số cần training: {num_trainable_elements}")

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Tổng số tham số       : {total_params:,}")
print(f"Số tham số cần training: {trainable_params:,}")

Số lượng tensor cần training: 32
Tổng số tham số cần training: 17062912
Tổng số tham số       : 60,241,984
Số tham số cần training: 17,062,912


In [ ]:
from torchsummary import summary
model = model.to("cuda" or "cpu")
summary(model, input_size=(3, 224, 224))  # Thay đổi input_size tùy vào model

### Read data

In [1]:
# read PDDD anotation
%store -r image_dict
def get_PDDD_info(image_id):
    img_info = image_dict[image_id]
    return img_info["file_name"]

%store -r category_dict
def get_PDDD_category(category_id):
    return category_dict[category_id]

%store -r image_paths labels

# from collections import Counter
# list = Counter(labels)

image_paths[:2], labels[:2], len(image_paths), len(labels)

(['/media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped/Tomato blight leaf/IMG_20221118_082850_368_jpg.rf.61b5ec99d23a3f0c5a1766e235fb3b6b_1086_Tomato blight leaf.jpg',
  '/media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped/Tomato blight leaf/IMG_20221118_081723_527_jpg.rf.337930f270075f587a13824248c1e5bd_3814_Tomato blight leaf.jpg'],
 ['Tomato blight leaf', 'Tomato blight leaf'],
 2765,
 2765)

## Run model

In [10]:
# common utils
from torchvision import transforms
import os
import matplotlib.pyplot as plt
from collections import Counter
from PIL import Image
import seaborn as sns
from tqdm import tqdm

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.416, 0.468, 0.355],   # normalize
                         std=[0.210, 0.206, 0.213])
])  # in PDDD paper

###############################    
def load_image(image_path):
    if image_path.startswith('http'):
        # Download image from URL
        response = requests.get(image_path)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        # Load local image
        image = Image.open(image_path).convert('RGB')
    return image

### Predict 

In [ ]:
# pretrained utils
def predict_plant(image_path):
    """Predict plant species and return class ID and confidence"""
    image = load_image(image_path)
    model.eval()
    
    # Make prediction
    with torch.no_grad():
        img_tensor = transform(image).unsqueeze(0).to(device)
        output = model(img_tensor)
        probs = torch.nn.functional.softmax(output[0], dim=0)
        confidence, pred_class = torch.topk(probs, 1)
    return pred_class.item(), confidence.item()

In [ ]:
# run prediction
pred_labels = []
confidences = []
for image_path in tqdm(image_paths[:10]):
    pred_class_id, confidence = predict_plant(image_path)
    # pred_label = get_PDDD_category(pred_class_id)
    pred_labels.append(pred_class_id)
    confidences.append(confidence)

pred_labels[:10], confidences[:10]

### Embedding

In [11]:
# embedding utils
import torch
from torchvision import models, transforms
from tqdm import tqdm
import faiss
import numpy as np
from PIL import Image

def model_forward(model_name, tensor):
    match model_name:
        case 'ConvNeXt.std' | 'eva02_large_patch14_clip_336.merged2b_ft_inat21':
            return model.forward_features(tensor)

        case _:
            output = model(tensor)
            embedding = features['pre_head']
            return embedding

def get_image_embedding(model, image, device='cuda'):
    img_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model_forward(model_name,img_tensor)  # lấy token embeddingput
        # embedding = torch.nn.functional.normalize(embedding, dim=1)
    return embedding.squeeze(0).cpu().numpy().reshape(1, -1)

def create_index(embeddings):
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index

def run_model(image_paths, labels):
    embeddings = []
    for path, label in tqdm(list(zip(image_paths, labels)), desc="Extract embeddings"):  # giới hạn 300 ảnh
        img = Image.open(path).convert("RGB")
        vec = get_image_embedding(model, img)
        embeddings.append(vec)
    labels = labels[:len(embeddings)]
    # hook_handle.remove()  
    return embeddings, labels

# Evaluate
def evaluate(index, labels, embeddings):
    correct = 0
    total = len(labels)
    pred_labels = []
    distances = []
    for i in range(total):
        top1, distance = inference(model,index,embeddings[i],i)
        pred_labels.append(labels[top1])
        distances.append(distance)
        if labels[i] == labels[top1]:
            correct += 1

    accuracy = correct / total
    return accuracy, pred_labels, distances

embeddings,labels = run_model(image_paths, labels)
embeddings_np = np.array(embeddings).astype("float32").squeeze(1)  # FAISS cần float32
index = create_index(embeddings_np)
embeddings_np.shape

Extract embeddings: 100%|██████████| 2765/2765 [01:14<00:00, 37.28it/s]


(2765, 2048)

In [12]:
# Evaluate accuracy
def inference(model,index,embedding, i):
        query_vec = embedding.reshape(1, -1)
        distance, result = index.search(query_vec, k=2)  # i: nearest vector, D: distance
        top1 = result[0][1] if result[0][0] == i else result[0][0]
        # top1 = result[0][0]
        return top1, distance

# Evaluate
def evaluate(index, labels, embeddings):
    correct = 0
    total = len(labels)
    pred_labels = []
    distances = []
    for i in tqdm(range(len(embeddings))):
        top1, distance = inference(model,index,embeddings[i],i)
        pred_labels.append(labels[top1])
        distances.append(distance)
        if labels[i] == labels[top1]:
            correct += 1

    accuracy = correct / len(embeddings)
    return accuracy, pred_labels, distances

accuracy, pred_labels, distances = evaluate(index, labels, embeddings_np)
accuracy, pred_labels[:3], distances[:2]

100%|██████████| 2765/2765 [00:02<00:00, 1003.42it/s]


(0.8491862567811935,
 ['Tomato Brown Spots', 'Tomato blight leaf', 'Tomato blight leaf'],
 [array([[  0.     , 129.12904]], dtype=float32),
  array([[ 0.      , 52.070724]], dtype=float32)])

### Test

In [ ]:
image = load_image('/media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped/Charbon_de_mais-1-_jpg.rf.d83e40544e3be2dac4fd66077d50344a_7246_Corn Smut.jpg')
embedding = get_image_embedding(model,image)
print(embedding.shape)

### File IO

In [ ]:
# init path
import os
import faiss
import numpy as np
import json
dir_path = '/media/icnlab/Data/Manh/tinyML/FieldPlant-11'
index_path = os.path.join(dir_path,f'{model_name}_trained.index')
json_path = os.path.join(dir_path,f'{model_name}_trained.json')

data_name = os.path.basename(dir_path)

In [9]:
# save file index
faiss.write_index(index, index_path)

# save class mapping (index -> class_id)
with open(json_path, "w") as f:
    json.dump(labels, f)

size_on_disk = os.path.getsize(index_path) / (1024 * 1024)
print(f"FAISS index file size: {size_on_disk:.2f} MB")
len(labels)

NameError: name 'index' is not defined

In [21]:
# Read file index
import faiss
import json
import os
# Đọc index từ file
index = faiss.read_index(index_path)
embeddings_np = index.reconstruct_n(0, index.ntotal)

# Đọc label mapping
with open(json_path, "r") as f:
    labels = json.load(f)  

size_on_disk = os.path.getsize(index_path) / (1024 * 1024)
print(f"FAISS index file size: {size_on_disk:.2f} MB")
model_name,data_name, embeddings_np.shape, len(labels)

FAISS index file size: 10.88 MB


('ConvNeXt.std', 'FieldPlant-11', (2786, 1024), 2786)